In [ ]:
fnm = "/home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/production_imgs/ALN_baoh_021.tif"
# fnm = "/home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/production_imgs/FeMoOx_AntiA_04_1k5x_CN.tif"

In [ ]:
import numpy as np
from PIL import Image
from PIL.TiffTags import TAGS
# print(TAGS)

In [ ]:
with Image.open(fnm, mode="r") as fp:
    for key in fp.tag_v2:
        if key in [34118, 34119]:
            print(type(fp.tag[key]))
            print(len(fp.tag[key]))        
            # print(f"{key}, {fp.tag[key]}")
        if key not in TAGS.keys():
            print(f"--->tag {key}, is not in PIL.TiffTAGS !")
    # self.tags = {TAGS[key] : fp.tag[key] for key in fp.tag_v2}
    # for key, val in self.tags.items():
    #     print(f"{key}, {val}")
    nparr = np.array(fp)
    print(f"{type(nparr)}")
    print(f"{nparr.dtype}")
    print(f"{np.shape(nparr)}")

In [ ]:
with Image.open(fnm, mode="r") as fp:
    czi_keys = [34118, 34119]
    for czi_key in czi_keys:
        if czi_key in fp.tag_v2:
            print(f"Found czi_key {tfs_key}...")
            utf = fp.tag[czi_key]
            print(type(utf))
            if len(utf) == 1:
                print(utf[0])
    # exit(1)
    tfs_keys = [34682]
    for tfs_key in tfs_keys:
        if tfs_key in fp.tag_v2:
            print(f"Found tfs_key {tfs_key}...")
            utf = fp.tag[tfs_key]
            print(type(utf))
            if len(utf) == 1:
                print(utf[0])

In [ ]:
import numpy as np
# https://www.geeksforgeeks.org/python-program-to-sort-a-list-of-tuples-by-second-item/
def sort_tuple(tup):
    # convert the list of tuples to a numpy array with data type (object, int)
    arr = np.array(tup, dtype=[('col1', object), ('col2', int)])
    # get the indices that would sort the array based on the second column
    indices = np.argsort(arr['col2'])
    # use the resulting indices to sort the array
    sorted_arr = arr[indices]
    # convert the sorted numpy array back to a list of tuples
    sorted_tup = [(row['col1'], row['col2']) for row in sorted_arr]
    return sorted_tup

In [ ]:
def if_str_represents_float(s):
    try:
        float(s)
        return str(float(s)) == s
    except ValueError:
        return False

In [ ]:
# TFS sections based on IKZ ALN_baoh_021.tif example
import mmap

tfs_section_names = ["[User]",
                     "[System]",
                     "[Beam]",
                     "[EBeam]",                 
                     "[GIS]",
                     "[Scan]",
                     "[EScan]",
                     "[Stage]",
                     "[Image]",
                     "[Vacuum]",
                     "[Specimen]",
                     "[Detectors]",
                     "[T2]",
                     "[Accessories]",
                     "[EBeamDeceleration]",
                     "[CompoundLensFilter]",
                     "[PrivateFei]",
                     "[HiResIllumination]",
                     "[EasyLift]",
                     "[HotStageMEMS]",
                     "[HotStage]",
                     "[HotStageHVHS]",
                     "[ColdStage]"]

tfs_section_details = {"[System]": ["Type", "Dnumber", "Software", "BuildNr", "Source", "Column", "FinalLens", "Chamber", "Stage", "Pump",
              "ESEM", "Aperture", "Scan", "Acq", "EucWD", "SystemType", "DisplayWidth", "DisplayHeight"]}
tfs_section_offsets = {}

with open(fnm, 'rb', 0) as file:
    s = mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ)
    for section_name in tfs_section_names:
        pos = s.find(bytes(section_name, "utf8"))  # != -1
        tfs_section_offsets[section_name] = pos
    print(tfs_section_offsets)

    # define search offsets
    tpl = []
    for key, value in tfs_section_offsets.items():
        tpl.append((key, value))
    # print(tpl)
    tpl = sort_tuple(tpl)
    print(tpl)
    # if section_name == "[System]":
    pos_s = None
    pos_e = None
    for idx in np.arange(0, len(tpl)):
        if tpl[idx][0] != "[System]":
            continue
        else:
            pos_s = tpl[idx][1]
            if idx <= len(tpl) - 1:
                pos_e = tpl[idx + 1][1]
            break
    print(f"Search in between byte offsets {pos_s} and {pos_e}")
    # fish metadata of e.g. the system section
    section_metadata = {}
    for term in tfs_section_details["[System]"]:
        
        s.seek(pos_s, 0)
        pos = s.find(bytes(term, "utf8"))
        if pos < pos_e:  # check if pos_e is None
            s.seek(pos, 0)
            section_metadata[f"{term}"] = f"{s.readline().strip().decode('utf8').replace(f'{term}=', '')}"
            if if_str_represents_float(section_metadata[f"{term}"]) is True:
                section_metadata[f"{term}"] = np.float64(section_metadata[f"{term}"])
            elif section_metadata[f"{term}"].isdigit() is True:
                section_metadata[f"{term}"] = np.int64(section_metadata[f"{term}"])
            else:
                pass
            # print(f"{term}, {pos}, {pos + len(term) + 1}")
    #        tfs_section_offswr
    #        file.seek(pos, 0)  #
    print(section_metadata)

In [ ]:
'2'.isdigit()